### Sample Script Prediction Script

Verify that loading and prediciton of the model is working correcticly

Hardware Setup:

    Memory (RAM): 31,4 GiB
    Processor - AMD® Ryzen 7 1700 eight-core processor × 16
    Graphics - GeForce GTX 1080 Ti/PCIe/SSE2
    OS type - 64-bit


In [ ]:
import mxnet as mx
import chess.variant
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'../../../')
import DeepCrazyhouse.src.runtime.Colorer
from DeepCrazyhouse.src.domain.util import *
from DeepCrazyhouse.src.domain.crazyhouse.input_representation import board_to_planes
from DeepCrazyhouse.src.domain.crazyhouse.output_representation import policy_to_moves
from DeepCrazyhouse.src.domain.agent.NeuralNetAPI import NeuralNetAPI
from DeepCrazyhouse.src.domain.crazyhouse.constants import *
from collections import deque
from time import time, sleep
%matplotlib inline

In [ ]:
ctx = 'cpu'

In [ ]:
net_api = NeuralNetAPI(ctx=ctx)

In [ ]:
net = net_api.get_net()

In [ ]:
board = chess.variant.CrazyhouseBoard()
board

In [ ]:
batch_size = 8
x = board_to_planes(board, board_occ=0, normalize=True)
x_mat_single = np.expand_dims(x, axis=0)
x_mat_batch = np.stack([x]*batch_size)

In [ ]:
x.shape

### Show the output prediction

In [ ]:
[value, policy] = net_api.predict_single(x)

In [ ]:
value

In [ ]:
LABELS[policy.argmax()]

In [ ]:
opts = 5
selected_moves, probs = policy_to_moves(board, policy)
selected_moves[:opts]

In [ ]:
plt.barh(range(opts)[::-1], probs[:opts])
ax = plt.gca()
ax.set_yticks(range(opts)[::-1])
ax.set_yticklabels(selected_moves)

In [ ]:
if ctx == 'cpu':
    ctx_mx = mx.cpu()
elif ctx == 'gpu':
    ctx_mx = mx.gpu()

In [ ]:
nb_batches = 500

### Runtime using batch_size of 1

In [ ]:
prediction_per_second = []
for it in range(7):
    print('Iteration %d' % (it+1))
    t_s = time()
    for i in range(nb_batches):
        net(mx.nd.array(x_mat_single, ctx=ctx_mx))

    elapsed_time = time() - t_s    
    print('elapsed time: %.3fs' % elapsed_time)
    preds_per_s = nb_batches / elapsed_time
    prediction_per_second.append(preds_per_s)
    print('predictions per second %.3f' % (preds_per_s))
print('Predictions/s: %.3f +/- %.3f' % (np.array(prediction_per_second).mean(), np.array(prediction_per_second).std()))    

### Runtime using given batch_size

In [ ]:
prediction_per_second = []
for it in range(7):
    print('Iteration %d' % (it+1))
    t_s = time()
    for i in range(nb_batches):
        net(mx.nd.array(x_mat_batch, ctx=ctx_mx))

    elapsed_time = time() - t_s    
    print('elapsed time: %.3fs' % elapsed_time)
    preds_per_s = (nb_batches*batch_size) / elapsed_time
    prediction_per_second.append(preds_per_s)
    print('predictions per second %.3f' % (preds_per_s))
print('Predictions/s: %.3f +/- %.3f' % (np.array(prediction_per_second).mean(), np.array(prediction_per_second).std()))    